In [2]:
import pandas as pd
from smartsheet_dataframe import get_as_df

In [3]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

In [4]:
# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)

In [5]:
war_df = mss_df.loc[mss_df['UniqueID'].str.contains("WAR")].copy()

In [6]:
client_df = war_df[['UniqueID',
                    'Tract',
                    'CMP',
                    'Stand',
                    'Acres',
                    'Age',
                    'County',
                    'State',
                    'Stand Type',
                    'Origin',
                    'Scheduled',
                    'PreBA',
                    'PreTPA',
                    'PreGTA',
                    'PlnBA',
                    'PlnTPA',
                    'PlnGTA',
                    'Client GT',
                    'Future GT',
                    'Reporting Period']].copy()

client_df['Reporting Period'] = client_df['Reporting Period'].fillna(0)
client_df.loc[client_df['Reporting Period'] == "", 'Reporting Period'] = 0
s = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
client_df['trial'] = client_df.index.map(s)
client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'] = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'trial']
client_df['Reporting Period'] = pd.to_numeric(client_df['Reporting Period'], downcast='integer')
client_df['Age'] = pd.to_numeric(client_df['Age'], errors='coerce', downcast='integer').fillna(0).astype(int)
client_df.loc[pd.to_numeric(client_df['Future GT'], errors='coerce').isnull(), 'Future GT'] = 0
client_df['Tract'] = client_df['Tract'].astype(str)
client_df['Stand'] = client_df['Stand'].astype(str)
client_df['Tract'] = client_df['Tract'].apply(lambda x: x.split('.')[0])
client_df['Client GT'] = pd.to_numeric(client_df['Client GT'], errors="coerce")
client_df['Client GT'] = client_df['Client GT'].fillna(0)

cols = ['PreBA', 'PreTPA', 'PreGTA', 'PlnBA', 'PlnTPA', 'PlnGTA']

for x in cols:
    client_df[x] = pd.to_numeric(client_df[x], errors='coerce', downcast='integer').fillna(-99).astype(int)

client_df.drop(columns=['trial'],inplace=True)


I am thinking of creating more tables : county, state, origin, species. That way if we have to update the way somethign is spelled we just do it in one place.

Maybe even tract as well?

In [27]:
species = ["Slash Pine", "Loblolly Pine", 'Mixed Natural Pine', 'Hardwood Mix', 'Pine Straw', "Other"]
species_dict = {'Species': species}
species_df = pd.DataFrame(species_dict)
species_df['SpeciesUID'] = species_df.index

def species_map(x):
    x = x.lower()
    if "slash" in x:
        return 0
    elif "loblol" in x:
        return 1
    elif "mixed natural" in x:
        return 2
    elif "h" in x:
        return 3
    elif "straw" in x:
        return 4
    elif "pine" in x:
        return 1
    else:
        return 5
    
client_df['Stand Type'] = client_df['Stand Type'].apply(lambda x: species_map(x))


In [37]:
origins = ['Planted', 'Natural', 'Natural Mix', 'Other']
origin_dict = {'Origin': origins}
origin_df = pd.DataFrame(origin_dict)
origin_df['OriginUID'] = origin_df.index

def origin_map(x):
    x = x.lower()
    if "plant" in x:
        return 0
    elif "natural mix" in x:
        return 2
    elif "nat" in x:
        return 1
    else:
        return 3
client_df['Origin'] = client_df['Origin'].apply(lambda x: origin_map(x))


AttributeError: 'int' object has no attribute 'lower'

In [7]:
ga_intake_df = war_df[['IntakeID',
                       'FolderID',
                       'UniqueID',
                       'GA_ACRES',
                       'GA GT', # this is going to be GA total isn't it? Or does it stay as GA estimate of current GT (total minus what client says is future)
                       'Total GT', # Does this need included? At the moment we do funny stuff with total gt and subtracting future gt to get GA current rp (above)
                       'Notes'
                       ]].copy()

# Do I want to include a Ticketid column here or use a crossreference table?
# Should reporting period be in here?

In [8]:
status_df = war_df[['UniqueID',
                    'Guidance',
                    'Expiration',
                    'GuidanceProvider',
                    'Status',
                    'Completed Date'
                    ]].copy()

In [9]:
def intake_breaker(lst):
    intake_lst = []
    activity_lst = []
    for ele in lst:
        intake_lst.append(ele[:9])
        activity_lst.append(ele[9:11])
        intake_st = set(intake_lst)
        activity_st = set(activity_lst)
        
    return intake_lst, activity_lst, intake_st, activity_st


def ticketid(lst):
    """This function creates a ticket id based off a list of unique ids

    Args:
        lst (list): a list of unique ids that have the same Tract_Cmp_Stand from MSS
    """
    if len(lst) == 0:
        return "ERROR, Where is your uniqueid?"
    elif len(lst) < 2:
        return lst[0]
    elif len(lst) > 1:
        # Check to see if there are multiple intakes
        temp_lst, activity_lst, temp_st, activity_st = intake_breaker(lst)
        temp_lst.sort()
        activity_lst.sort()
        if len(temp_st) > 1:
            return f"{temp_lst[0]}XXXX"
        # If there are not multiple intakes. Check to see if there are multiple activities in the lst (Intake portion should be same for all)
        elif len(activity_st) > 1:
           return f"{temp_lst[0]}XXXX"
       # If there is only a single activity type
        elif len(activity_st) == 1:
            return f"{temp_lst[0]}{activity_lst[0]}XX"
        else:
            return "Failed within len(lst) > 1"
        
    else:
        return "Failed at length of lst" 

In [10]:
war_inprog = war_df[war_df['Status'].isin(['OPEN', 'IN PROCESS', 'POST', 'PENDING'])].copy()
war_inprog['Tract'] = war_inprog['Tract'].astype(str).apply(lambda x: x.split('.')[0])
war_inprog['CMP'] = war_inprog['CMP'].astype(str).apply(lambda x: x.split('.')[0])
war_inprog['Stand'] = war_inprog['Stand'].astype(str).apply(lambda x: x.split('.')[0])
war_inprog['TCS'] = war_inprog['Tract'] + "_" + war_inprog['CMP'] + "_" + war_inprog['Stand']

# After talking with Rob, we need to make sure that we separate out the GA and the ARB Guidance
grouped_activities = war_inprog.groupby(['TCS', 'GuidanceProvider'])['UniqueID'].apply(list).reset_index()

trial_df = grouped_activities.copy()
trial_df['BBID'] = trial_df['UniqueID'].apply(lambda x: ticketid(x))

prev_combined = war_inprog.groupby('TicketID')['TCS'].apply(set).reset_index()

tcs = []
for val in prev_combined['TCS'].loc[prev_combined['TCS'].str.len()> 1].values:
    for item in val:
        tcs.append(item)
        
ticket_id_dict = {}
for i, row in prev_combined.loc[prev_combined['TCS'].str.len()> 1].iterrows():
    # print(row['TicketID']) # ticket id
    value_list = []
    for value in row['TCS']:
        value_list.append(value)
    ticket_id_dict[row['TicketID']] = value_list
    
for k, v in ticket_id_dict.items():
    trial_df.loc[trial_df['TCS'].isin(v), 'otherid'] = k

def final_id(row):
    if row['otherid'] != "nan":
        return row['otherid']
    else:
        return row['BBID']
    
trial_df['finalid'] = trial_df[['BBID', 'otherid']].apply(final_id, axis=1)

def break_list(x):
    trial_list = []
    for item in x:
        for iter in item:
            trial_list.append(iter)
            
    return trial_list

new_ticket_df = trial_df.groupby('finalid').agg({'TCS': lambda x: list(x),
                                 'UniqueID': lambda x: break_list(x)}).reset_index()

client_for_ticket = client_df.copy()
for id in new_ticket_df['finalid'].unique().tolist():
    client_for_ticket.loc[client_for_ticket['UniqueID'].isin(new_ticket_df.loc[new_ticket_df['finalid'] == id, 'UniqueID'].values[0]), 'TicketID'] = id

new_tcs_intake = client_for_ticket.groupby(['TicketID',
                                            'UniqueID'])['Client GT'].sum().reset_index()

old_ticket_ids = dict(zip(war_df['UniqueID'], war_df['TicketID']))
new_tcs_intake['old_ticketids'] = new_tcs_intake['UniqueID'].map(old_ticket_ids)
new_tcs_intake.loc[~new_tcs_intake['TicketID'].str.contains("WAR"), 'TicketID'] = new_tcs_intake['old_ticketids']

ticketid_table = new_tcs_intake[['TicketID', 'UniqueID']]

# Getting the expected harvest

Find all activities that are Open, Pending, In Process, Post

Get their ticket id

Get the sum of the expected rp8 for that ticketid
get the sum of the expected rp9 for that ticketid

get the harvested volumes for those ticketids (already did this by getting the sums and translating from old ticketid to new ticketid)